In [2]:
import pandas as pd
import os

Opened the .xlsx file with the merged data:

In [3]:
data = pd.read_excel("30_03_20_speeded_naming_results.xlsx")

In [4]:
data.tail()

,ID,Gender,Age,Trail_BA,Block,Rand,Order,CatPos,Sequence,Item,...,Familiarity,Length,NameAgr,VisCom,AoA,Imageabil,ImageAgr,NumFeats,Typicality,NearSemNeigh
24943,180,F,20,2.221066,3,6,163,5,NaN,180wand,...,2.60,4,0.818182,2.090909,3.083333,5.500000,3.703704,9,15.000000,0
24944,180,F,20,2.221066,2,6,51,1,NaN,180whale,...,3.50,3,0.818182,2.272727,2.818182,5.454545,4.727273,10,35.545455,13
24945,180,F,20,2.221066,2,6,57,2,NaN,180whip,...,3.75,3,1.000000,1.818182,5.181818,5.727273,4.551724,13,23.400000,1
24946,180,F,20,2.221066,2,6,100,3,NaN,180whistle,...,6.40,4,1.000000,1.909091,4.545455,4.727273,4.461538,14,24.600000,1
24947,180,F,20,2.221066,3,6,161,16,NaN,180zebra,...,2.60,5,1.000000,2.181818,3.090909,6.181818,4.777778,13,31.453125,3


Set Item as index:

In [5]:
data.set_index('Item', inplace=True)

In [56]:
data.head()

,ID,Gender,Age,Trail_BA,Block,Rand,Order,CatPos,Sequence,Stimulus,...,Familiarity,Length,NameAgr,VisCom,AoA,Imageabil,ImageAgr,NumFeats,Typicality,NearSemNeigh
Item,,,,,,,,,,,,,,,,,,,,,
105ambulance,105,F,19,1.974596,4,1,291,17,3.0,ambulance,...,6.45,9,1.000000,2.636364,3.363636,5.909091,3.785714,17,23.827586,0
105anchor,105,F,19,1.974596,1,1,2,0,3.0,anchor,...,3.85,5,1.000000,1.818182,3.636364,5.272727,4.384615,13,19.875000,2
105ant,105,F,19,1.974596,2,1,77,2,3.0,ant,...,7.55,3,0.833333,2.583333,1.909091,6.181818,4.037037,11,34.428571,3
105apple,105,F,19,1.974596,3,1,178,7,3.0,apple,...,8.30,3,1.000000,1.090909,2.000000,6.636364,4.920000,17,61.730769,22
105apron,105,F,19,1.974596,4,1,271,14,3.0,apron,...,4.75,5,1.000000,1.909091,3.272727,5.363636,3.840000,13,11.000000,0


In [703]:
# we can also subset each participant by the value of the column "ID":
#def subset_one_person(IDnum):
    #person  = data[data.ID == IDnum]
    #return person

Processing each .txt file with each participant:

In [8]:
def adding_ID(filename):
    file = pd.read_csv("./MyFinalParticipants/" + filename, sep = "\t", error_bad_lines=False) #removed "101_results_pp.txt"
    file = file.reset_index()
    file.columns = ["rater", "RT", "RT-100", "ACC", "Type", "date", "time", "Response", "soundfile"]
    filename = filename.split("_")
    file["participant"] = int(filename[0])
    return file

In [9]:
def delete_practice_trials(file):
    practice_trials = file[file['soundfile'].astype(str).str.startswith("p_")]
    result = file.drop(practice_trials.index)
    return result

In [63]:
def preprocessing_txt(filename):
    withID = adding_ID(filename)
    result = delete_practice_trials(withID)
    result["Item"] = result["participant"].apply(str) + result["soundfile"].str.split("-").str[0]
    #result["participant"].apply(str) 
    result.set_index('Item', inplace=True)
    new = result[["participant", 'RT', 'RT-100', 'ACC', "Type", "Response"]].copy() #selected 5 columns to fill in big df
    return new

In [39]:
file_IDs = list(data.ID.values)
sorted_data_IDs = sorted(set(file_IDs), key=file_IDs.index)#array with sorted data indices: 105, 108, 109, 110, 119..
sorted_data_IDs.remove(164)

In [41]:
def duplicates_stimuli():
    output = ""
    for ind in sorted_data_IDs:
        #except FileNotFoundError:
        person = preprocessing_txt(str(ind) + "_results_pp.txt")
        if len(person.index.get_duplicates()) > 0:
            output += " ".join(person.index.get_duplicates()) + " "
    return output

duplicates_stimuli()
#sorted_data_IDs

C:\Users\Omistaja\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: 'get_duplicates' is deprecated and will be removed in a future release. You can use idx[idx.duplicated()].unique() instead
  
C:\Users\Omistaja\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'get_duplicates' is deprecated and will be removed in a future release. You can use idx[idx.duplicated()].unique() instead
  import sys
b'Skipping line 171: expected 9 fields, saw 10\n'
b'Skipping line 147: expected 9 fields, saw 10\nSkipping line 253: expected 9 fields, saw 10\n'
b'Skipping line 208: expected 9 fields, saw 10\n'
b'Skipping line 90: expected 9 fields, saw 10\n'
b'Skipping line 225: expected 9 fields, saw 10\n'


'110ambulance 110anchor 124strawberry 171ambulance 171anchor '

In [42]:
duplicates_stimuli()

C:\Users\Omistaja\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: 'get_duplicates' is deprecated and will be removed in a future release. You can use idx[idx.duplicated()].unique() instead
  
b'Skipping line 171: expected 9 fields, saw 10\n'
b'Skipping line 147: expected 9 fields, saw 10\nSkipping line 253: expected 9 fields, saw 10\n'
b'Skipping line 208: expected 9 fields, saw 10\n'
b'Skipping line 90: expected 9 fields, saw 10\n'
b'Skipping line 225: expected 9 fields, saw 10\n'


''

In [58]:
part101 = preprocessing_txt("110_results_pp.txt")
#part101["stimulusID"] = part101["participant"].apply(str) + part101["soundfile"].str.split("-").str[0]
#part101
#compounds=part101.Item.str[1].str.isalpha()
#part101[compounds]


In [75]:
prefinal = pd.read_excel("15prefinal.xlsx")
prefinal.set_index('Item', inplace=True)
prefinal.head()

,ID,Gender,Age,Trail_BA,Block,Rand,Order,CatPos,Sequence,Stimulus,...,Familiarity,Length,NameAgr,VisCom,AoA,Imageabil,ImageAgr,NumFeats,Typicality,NearSemNeigh
Item,,,,,,,,,,,,,,,,,,,,,
105ambulance,105,F,19,1.974596,4,1,291,17,3.0,ambulance,...,6.45,9,1.000000,2.636364,3.363636,5.909091,3.785714,17,23.827586,0
105anchor,105,F,19,1.974596,1,1,2,0,3.0,anchor,...,3.85,5,1.000000,1.818182,3.636364,5.272727,4.384615,13,19.875000,2
105ant,105,F,19,1.974596,2,1,77,2,3.0,ant,...,7.55,3,0.833333,2.583333,1.909091,6.181818,4.037037,11,34.428571,3
105apple,105,F,19,1.974596,3,1,178,7,3.0,apple,...,8.30,3,1.000000,1.090909,2.000000,6.636364,4.920000,17,61.730769,22
105apron,105,F,19,1.974596,4,1,271,14,3.0,apron,...,4.75,5,1.000000,1.909091,3.272727,5.363636,3.840000,13,11.000000,0


In [76]:
#train = #110,119,122]#124,133,136,147,177,178,106,
         #107,117,118,121,131,132,134,135,145,146,148,159,160,149,
        # 150,157,158,162,163,164,170,171,172,173,176,181,103,104,
         #111,112,161,169,174,175,182,183,101,102,114,115,116,
         #128,129,#130,#139,142,143,144,153,156,167,113,126,127,137,138,140,141,151,152,154,155,165,166,168,179,180]
damaged = [165]
for ind in damaged:
    one_participant = preprocessing_txt(str(ind) + "_results_pp.txt")
    #print(one_participant.head())
    prefinal.update(one_participant)

In [77]:
prefinal.to_excel("07_04_20_speeded_naming_resultsFinaltable.xlsx")  

In [ ]:
part110 =  preprocessing_txt("_results_pp.txt")

In [74]:
data["RT"][1060:1100]

Item
110motorcycle         -1.0
110mouse              -1.0
110mouse_computer    839.0
110mug                 NaN
110mushroom            NaN
110napkin              NaN
110octopus             NaN
110olive               NaN
110onions              NaN
110orange              NaN
110otter               NaN
110oven                NaN
110owl                 NaN
110pants               NaN
110peach               NaN
110peacock             NaN
110pear                NaN
110peg                 NaN
110pelican             NaN
110pen                 NaN
110pencil              NaN
110penguin             NaN
110piano               NaN
110pickle              NaN
110pie                 NaN
110pig                 NaN
110pigeon              NaN
110pillow              NaN
110pineapple           NaN
110pipe_plumbing       NaN
110pipe_smoking        NaN
110plate               NaN
110platypus            NaN
110plug_electric       NaN
110pony                NaN
110pot                 NaN
110potato              

In [62]:
part105=preprocessing_txt("111_results_pp.txt")
data.update(part105)

In [421]:
df = pd.DataFrame({'ID': ["1a_a-S-S", "1b_a-S-S", "1c_a-S-S", "3a_a-S-S", "3b_a-S-S", "3c_a-S-S", "4a_a-S-S", "4b_a-S-S", "4c_a-S-S", "0a-S-S", "0b-S-S", "0d-S-S"],
                   'A': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                   'B': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120],
                   'C': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200],})

df1 = pd.DataFrame({'ID': ["1a", "1b", "1c"],
                    'B': ["jan", "feb", "mar"],
                    'C': [1997, 1996, 1996]})

df2 = pd.DataFrame({'ID': ["3a", "3b", "3c"],
                    'B': ["apr", "may", "jun"],
                    'C': [1987, 1997, 1996]})
df3 = pd.DataFrame({'ID': ["4a", "4b", "4c"],
                    'B': ["jul", "aug", "sep"],
                    'C': [1777, 1917, 1196]})
df4 = pd.DataFrame({'ID': ["0a", "0b", "0d"],
                    'B': ["oct", "nov", "dec"],
                    'C': [1984, 1947, 1496]})

In [422]:
df

,ID,A,B,C
0,1a_a-S-S,1,10,100
1,1b_a-S-S,2,20,200
2,1c_a-S-S,3,30,300
3,3a_a-S-S,4,40,400
4,3b_a-S-S,5,50,500
5,3c_a-S-S,6,60,600
6,4a_a-S-S,7,70,700
7,4b_a-S-S,8,80,800
8,4c_a-S-S,9,90,900
9,0a-S-S,10,100,1000


In [420]:
#df.set_index('ID', inplace=True)
#df1.set_index('ID', inplace=True)
#df2.set_index('ID', inplace=True)
#df3.set_index('ID', inplace=True)
#df4.set_index('ID', inplace=True)

In [426]:
df["FIRST"] = df["ID"].str.split("-").str[0]

In [428]:
df.set_index('FIRST', inplace=True)

In [386]:
df.update(df4)

In [293]:
df.loc[df.ID.isin(df1.ID), ['B', 'C']] = df1[['B', 'C']]

In [298]:
df

,ID,A,B,C
0,1a,1,jan,1997.0
1,1b,2,feb,1996.0
2,1c,3,mar,1996.0
3,3a,4,NaN,NaN
4,3b,5,NaN,NaN
5,3c,6,NaN,NaN
6,4a,7,70,700.0
7,4b,8,80,800.0
8,4c,9,90,900.0
9,0a,10,NaN,NaN
